In [1]:
from glob import glob
import nibabel as nib, shutil as sht
from tqdm.notebook import tqdm
from tkinter import filedialog
import tkinter as TK, os
import pandas as pd

In [2]:
def mkdir(path):
    try: os.mkdir(path)
    except: pass

def create_runner(path, subject, folder_name=-1):
    if    folder_name==-1: 
        FILE = open(path+'/Run_'+subject+'.m','w')
        job_path = path+'/'+'/job_'+subject+'.m'
    else: 
        mkdir(path+'/'+folder_name); FILE = open(path+'/'+folder_name+'/Run_'+subject+'.m','w')
        job_path = path+'/'+folder_name+'/job_'+subject+'.m'

    FILE.write("""%% List of open inputs
nrun = 1; %% enter the number of runs here
jobfile = {'%s'};
jobs = repmat(jobfile, 1, nrun);
inputs = cell(0, nrun);
for crun = 1:nrun
end
spm('defaults', 'FMRI');
spm_jobman('run', jobs, inputs{:});
"""%(job_path))
    FILE.close()
    JOB_FILE = open(job_path,'w')
    
    return JOB_FILE

In [3]:
def Event_loader(path, event_names=-1): # event_names = ['lf', 'lh'] 등으로 지정하면 해당 파일만 불러옵니다.
    events = {}
    if event_names!=-1:
        for ev in event_names:
            events[ev] = [[],[]]
            with open(path+'/{}.txt'.format(ev), 'r') as f:
                for line in f.readlines(): 
                    d = line.split('\t')
                    events[ev][0].append(float(d[0]))
                events[ev][1].append(float(d[1]))
                if len(events[ev][0])==0 or len(events[ev][1])==0:  print('[Warning]',path, ev, 'is not normal event file!' ); del events[ev]

    else: #event_names=-1이면 이벤트 폴더에 있는 모든 파일을 불러옵니다. 단, Sync는 제외합니다.
        for ev in sorted(glob(path+'/*.txt')): 
            if 'Sync' in ev: continue
            event = ev.split('/')[-1].replace('.txt',''); events[event] = [[],[]]
            with open(ev,'r') as f: 
                for line in f.readlines(): 
                    d = line.split('\t')
                    events[event][0].append(float(d[0]))
                events[event][1].append(float(d[1]))
                if len(events[event][0])==0 or len(events[event][1])==0:  print('[Warning]',path, ev, 'is not normal event file!' ); del events[event]
       
    
    return events


In [4]:
Use_RL_LR = 2 #Phase direction -> 1: RL, 2: LR

In [5]:
PATH        = '/data/HCP/tfMRI/WM' #데이터 폴더
EVENT_PATH  = '/MNINonLinear/Results/tfMRI_WM_*/EVs' #EVENT 파일이 저장된 경로를 지정하세요.
Script_PATH = '/data/HCP/tfMRI/HCP_SCRIPT_6mm/WM'  #스크립트가 저장될 폴더 -> Subject 별로 스크립트 생성
Output_PATH = '/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM'  #전처리한 데이터가 저장될 경로

In [6]:
nii_gz_to_nii  = 0 # SPM에서는 .gz 파일을 불러오지 못하므로 True로 하면 *.nii.gz파일을 전부 압축해제 합니다.
if nii_gz_to_nii:
    os.chdir(PATH); os.system('gzip -d -r *') #주의! 원본 gz파일이 전부 제거됩니다.

In [6]:
df = pd.DataFrame()
df['Subject'], df['task_file'], df['art_file'], df['phase_direction'] = None, None, None, None
idx = 0
for subject in sorted(glob(PATH+'/*'),key=lambda x: x.split('/')[-1]):
    print(subject)
    if os.path.isdir(subject)==False: continue

    task_files = [ glob(subject+'/MNINonLinear/Results/tfMRI_*_{0}/tfMRI_*_{0}.nii'.format('RL' if Use_RL_LR==1 else 'LR'))[0] ]
    art_files  = [ glob(subject+'/MNINonLinear/Results/tfMRI_*_{0}/Movement_Regressors.txt'.format('RL' if Use_RL_LR==1 else 'LR'))[0]  ]

    for Type, File in enumerate(zip(task_files, art_files)):
        df.loc[idx] =  [subject.split('/')[-1], File[0], File[1], 'RL' if 'RL' in File[0] else 'LR']
        idx += 1

/data/HCP/tfMRI/WM/100206
/data/HCP/tfMRI/WM/100307
/data/HCP/tfMRI/WM/100408
/data/HCP/tfMRI/WM/100610
/data/HCP/tfMRI/WM/101006
/data/HCP/tfMRI/WM/101107
/data/HCP/tfMRI/WM/101309
/data/HCP/tfMRI/WM/101915
/data/HCP/tfMRI/WM/102109
/data/HCP/tfMRI/WM/102513
/data/HCP/tfMRI/WM/102614
/data/HCP/tfMRI/WM/102715
/data/HCP/tfMRI/WM/102816
/data/HCP/tfMRI/WM/103010
/data/HCP/tfMRI/WM/103111
/data/HCP/tfMRI/WM/103212
/data/HCP/tfMRI/WM/103414
/data/HCP/tfMRI/WM/103818
/data/HCP/tfMRI/WM/104416
/data/HCP/tfMRI/WM/105014
/data/HCP/tfMRI/WM/105115
/data/HCP/tfMRI/WM/105216
/data/HCP/tfMRI/WM/105620
/data/HCP/tfMRI/WM/105923
/data/HCP/tfMRI/WM/106016
/data/HCP/tfMRI/WM/106319
/data/HCP/tfMRI/WM/106521
/data/HCP/tfMRI/WM/106824
/data/HCP/tfMRI/WM/107018
/data/HCP/tfMRI/WM/107321
/data/HCP/tfMRI/WM/107422
/data/HCP/tfMRI/WM/108020
/data/HCP/tfMRI/WM/108121
/data/HCP/tfMRI/WM/108222
/data/HCP/tfMRI/WM/109123
/data/HCP/tfMRI/WM/109830
/data/HCP/tfMRI/WM/110007
/data/HCP/tfMRI/WM/110411
/data/HCP/tf

In [7]:
df

,Subject,task_file,art_file,phase_direction
0,100206,/data/HCP/tfMRI/WM/100206/MNINonLinear/Results...,/data/HCP/tfMRI/WM/100206/MNINonLinear/Results...,LR
1,100307,/data/HCP/tfMRI/WM/100307/MNINonLinear/Results...,/data/HCP/tfMRI/WM/100307/MNINonLinear/Results...,LR
2,100408,/data/HCP/tfMRI/WM/100408/MNINonLinear/Results...,/data/HCP/tfMRI/WM/100408/MNINonLinear/Results...,LR
3,100610,/data/HCP/tfMRI/WM/100610/MNINonLinear/Results...,/data/HCP/tfMRI/WM/100610/MNINonLinear/Results...,LR
4,101006,/data/HCP/tfMRI/WM/101006/MNINonLinear/Results...,/data/HCP/tfMRI/WM/101006/MNINonLinear/Results...,LR
...,...,...,...,...
769,991267,/data/HCP/tfMRI/WM/991267/MNINonLinear/Results...,/data/HCP/tfMRI/WM/991267/MNINonLinear/Results...,LR
770,992774,/data/HCP/tfMRI/WM/992774/MNINonLinear/Results...,/data/HCP/tfMRI/WM/992774/MNINonLinear/Results...,LR
771,993675,/data/HCP/tfMRI/WM/993675/MNINonLinear/Results...,/data/HCP/tfMRI/WM/993675/MNINonLinear/Results...,LR
772,994273,/data/HCP/tfMRI/WM/994273/MNINonLinear/Results...,/data/HCP/tfMRI/WM/994273/MNINonLinear/Results...,LR


In [8]:
for Subject_info in df.loc[0]:
    print(Subject_info)

100206
/data/HCP/tfMRI/WM/100206/MNINonLinear/Results/tfMRI_WM_LR/tfMRI_WM_LR.nii
/data/HCP/tfMRI/WM/100206/MNINonLinear/Results/tfMRI_WM_LR/Movement_Regressors.txt
LR


## Generate output folder per subject

In [10]:
mkdir(Output_PATH); mkdir(Script_PATH)
for idx in tqdm(  range(len(df)) ):
    subject, task, art, Pd  = df.loc[idx]
    mkdir(Output_PATH+'/'+subject)

  0%|          | 0/774 [00:00<?, ?it/s]

## Model specification and Estimation

In [9]:
First_Model_Parameter = {
    'Time_Units': 'secs',
    'TR': 0.72,
    'Event_names' : ['0bk_body', '0bk_cor','0bk_faces', '0bk_places', '0bk_tools', '2bk_body', '2bk_cor', '2bk_faces', '2bk_places', '2bk_tools', 'all_bk_cor'], #-1로 설정하면 모든 이벤트 파일을 로드합니다.
    'Total_Volume': 405 # 'Auto': 파일 별로 volume을 계산하여 기입합니다. 다소 시간이 걸립니다.  /  숫자: 모든 volume의 개수를 알고 그 개수가 같을 때 사용합니다.
}

In [16]:
for idx in tqdm(  range(len(df)) ):
    subject, task, art, Pd = df.loc[idx]
    try: src_file   = nib.load(task).get_fdata()
    except OSError:
        print('Subject #{}  was not loaded!'.format(subject)); continue
    event_info = Event_loader(PATH+'/{0}{1}'.format(subject, EVENT_PATH.replace('*',Pd))
                                , event_names=First_Model_Parameter['Event_names'])

    mkdir(Output_PATH+'/'+subject)

    JOB_FILE = create_runner(Script_PATH, subject, '1st_model')
    
    JOB_FILE.write("matlabbatch{1}.spm.spatial.smooth.data = {\n")
    for slice in range(1,(1+src_file.shape[-1]) if First_Model_Parameter['Total_Volume']=='Auto' else (1+First_Model_Parameter['Total_Volume'])):
        JOB_FILE.write("                                                    '%s,%d'\n"%(task,slice))
    JOB_FILE.write("                                                    };\n")

    JOB_FILE.write("matlabbatch{1}.spm.spatial.smooth.fwhm = [6 6 6];\n")
    JOB_FILE.write("matlabbatch{1}.spm.spatial.smooth.dtype = 0;\n")
    JOB_FILE.write("matlabbatch{1}.spm.spatial.smooth.im = 0;\n")
    JOB_FILE.write("matlabbatch{1}.spm.spatial.smooth.prefix = 's';\n")
       

    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.dir = {'%s'};\n"       %(Output_PATH+'/'+subject))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.timing.units = '%s';\n"%(First_Model_Parameter['Time_Units']))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.timing.RT = %.2f;\n"   %(First_Model_Parameter['TR']))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.timing.fmri_t = %d;\n" %(16))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.timing.fmri_t0 = %d;\n"%(8))
    
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.scans = {\n")
    for slice in range(1,(1+src_file.shape[-1]) if First_Model_Parameter['Total_Volume']=='Auto' else (1+First_Model_Parameter['Total_Volume'])):
        JOB_FILE.write("                                                    's%s,%d'\n"%(task,slice))
    JOB_FILE.write("                                                    };\n")

    for event_idx, sess in enumerate( event_info.keys() ):
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).name = '%s';\n"%(event_idx+1, sess))
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).onset = [\n"%(event_idx+1))
        for onset in event_info[sess][0]: 
            JOB_FILE.write("                                                          %f\n"%(onset))
        JOB_FILE.write("                                                          ];\n")
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).duration = %f\n"%(event_idx+1, event_info[sess][1][0]))
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).tmod = 0;\n"%(event_idx+1))
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).pmod = struct('name', {}, 'param', {}, 'poly', {});\n"%(event_idx+1))
        JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.cond(%d).orth = 1;\n"%(event_idx+1))

    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.multi = {''};\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.regress = struct('name', {}, 'val', {});\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.multi_reg = {'%s'};\n"%(art))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.sess.hpf = %d;\n"%(128))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.fact = struct('name', {}, 'levels', {});\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.bases.hrf.derivs = [0 0];\n") #Time derivative, Dispersion derivative
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.volt = 1;\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.global = 'None';\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.mthresh = %s;\n"%(0.8))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.mask = {''};\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_spec.cvi = 'AR(1)';\n")

    JOB_FILE.write("matlabbatch{3}.spm.stats.fmri_est.spmmat = {'%s'};\n"%(Output_PATH+'/'+subject+'/SPM.mat'))
    JOB_FILE.write("matlabbatch{3}.spm.stats.fmri_est.write_residuals = 0;\n")
    JOB_FILE.write("matlabbatch{3}.spm.stats.fmri_est.method.Classical = 1;\n")

    JOB_FILE.close()

  0%|          | 0/774 [00:00<?, ?it/s]

### Output Check - `1st Model`

In [10]:
subject, task, art, Pd = df.loc[0]
event_info = Event_loader(PATH+'/{0}{1}'.format(subject, EVENT_PATH.replace('*',Pd)), event_names=First_Model_Parameter['Event_names'])
event_info.keys()

dict_keys(['0bk_body', '0bk_cor', '0bk_faces', '0bk_places', '0bk_tools', '2bk_body', '2bk_cor', '2bk_faces', '2bk_places', '2bk_tools', 'all_bk_cor'])

In [11]:
Output_PATH

'/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM'

In [12]:
df2 = pd.DataFrame()
df2['Subject'], df2['Path'], df2['SPM.mat'], df2['beta'] = None, None, None, None
for idx, Subject in enumerate(sorted(glob(Output_PATH+'/*'), key=lambda x: x.split('/')[-1] )):
    ID, SPM, beta = Subject.split('/')[-1], glob(Subject+'/SPM.mat')[0], len(glob(Subject+'/beta*.nii'))
    if beta==0: print('Check {}'.format(Subject))
    df2.loc[idx] = [ID, Subject, SPM, beta]

In [13]:
df2

,Subject,Path,SPM.mat,beta
0,100206,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100206,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100206/SPM.mat,24
1,100307,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100307,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100307/SPM.mat,24
2,100408,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100408,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100408/SPM.mat,24
3,100610,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100610,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/100610/SPM.mat,24
4,101006,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/101006,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/101006/SPM.mat,24
...,...,...,...,...
769,991267,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/991267,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/991267/SPM.mat,24
770,992774,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/992774,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/992774/SPM.mat,24
771,993675,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/993675,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/993675/SPM.mat,24
772,994273,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/994273,/data/HCP/tfMRI/HCP_OUTPUT_6mm/WM/994273/SPM.mat,24


## Contrast Manager

In [14]:
def Select_Contrast(Pos_idx, Neg_idx,keys=list(event_info.keys())): 
    contrast_name = '{0}-{1}'.format(keys[Pos_idx], keys[Neg_idx])
    contrast_arry = '[ '
    for i in range(len(keys)):
        if    i==Pos_idx: contrast_arry += ' 1 '
        elif  i==Neg_idx: contrast_arry += '-1 '
        else:             contrast_arry += ' 0 '
    contrast_arry += ']'

    print("%-25s"%(contrast_name),'-->  ', contrast_arry)

    return contrast_name, contrast_arry

In [15]:
for idx, key in enumerate( list(event_info.keys()) ):
    print(idx, key)

0 0bk_body
1 0bk_cor
2 0bk_faces
3 0bk_places
4 0bk_tools
5 2bk_body
6 2bk_cor
7 2bk_faces
8 2bk_places
9 2bk_tools
10 all_bk_cor


In [17]:
Contrast_list = [ # Contrast[Pos_idx] - Contrast[Neg_idx]
    Select_Contrast(Pos_idx=5, Neg_idx=0),
    Select_Contrast(Pos_idx=7, Neg_idx=2),
    Select_Contrast(Pos_idx=8, Neg_idx=3),
    Select_Contrast(Pos_idx=9, Neg_idx=4),
    Select_Contrast(Pos_idx=6, Neg_idx=1)
]

2bk_body-0bk_body         -->   [ -1  0  0  0  0  1  0  0  0  0  0 ]
2bk_faces-0bk_faces       -->   [  0  0 -1  0  0  0  0  1  0  0  0 ]
2bk_places-0bk_places     -->   [  0  0  0 -1  0  0  0  0  1  0  0 ]
2bk_tools-0bk_tools       -->   [  0  0  0  0 -1  0  0  0  0  1  0 ]
2bk_cor-0bk_cor           -->   [  0 -1  0  0  0  0  1  0  0  0  0 ]


In [18]:
Contrast = {}
for con in Contrast_list: Contrast[con[0]] = con[1]

In [19]:
for idx in tqdm(  range(len(df)) ):
    subject, task, art, _ = df.loc[idx]
    JOB_FILE = create_runner(Script_PATH, subject,'1st_contrast')
    JOB_FILE.write("matlabbatch{1}.spm.stats.con.spmmat = {'%s'};\n"%(Output_PATH+'/'+subject+'/SPM.mat'))
    for Idx, contrast in enumerate(Contrast):
        JOB_FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.name = '%s';\n"%(Idx+1,contrast) )
        JOB_FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.weights = %s;\n"%(Idx+1, Contrast[contrast]) )
        JOB_FILE.write("matlabbatch{1}.spm.stats.con.consess{%d}.tcon.sessrep = 'none';\n"%(Idx+1) )
    JOB_FILE.write("matlabbatch{1}.spm.stats.con.delete = 1;\n")
    JOB_FILE.close()

  0%|          | 0/774 [00:00<?, ?it/s]

### Output Check - `Contrast`

In [25]:
df3 = pd.DataFrame()
df3['Subject'], df3['Path'], df3['con'], df3['spmT'] = None, None, None, None
for idx, Subject in enumerate(sorted(glob(Output_PATH+'/*'), key=lambda x: x.split('/')[-1] )):
    ID, con, spmT = Subject.split('/')[-1], len(glob(Subject+'/con*.nii')), len(glob(Subject+'/spmT*.nii'))
    if con!=len(Contrast_list) or spmT!=len(Contrast_list): print('Please Check {0} - index: {1}'.format(Subject, idx))
    df3.loc[idx] = [ID, Subject, con, spmT]

In [26]:
df3

,Subject,Path,con,spmT
0,100206,/data/kyh/Workspace/WM0/Output/100206,5,5
1,100307,/data/kyh/Workspace/WM0/Output/100307,5,5
2,100408,/data/kyh/Workspace/WM0/Output/100408,5,5
3,100610,/data/kyh/Workspace/WM0/Output/100610,5,5
4,101006,/data/kyh/Workspace/WM0/Output/101006,5,5
...,...,...,...,...
769,991267,/data/kyh/Workspace/WM0/Output/991267,5,5
770,992774,/data/kyh/Workspace/WM0/Output/992774,5,5
771,993675,/data/kyh/Workspace/WM0/Output/993675,5,5
772,994273,/data/kyh/Workspace/WM0/Output/994273,5,5


## Second level model
### Model specification + Model estimation + Contrast manager

In [31]:
Second_level_output = '/data/kyh/Workspace/WM0/Output_2nd'; mkdir(Second_level_output); mkdir(Script_PATH+'/1sample')
Mode = 'one_sample' #one_sample, two_sample

In [28]:
Script_PATH, Output_PATH

('/data/kyh/Workspace/WM0/Script', '/data/kyh/Workspace/WM0/Output')

In [29]:
def One_Sample_t_TEST(flist, script_path, output_path, contrast_one ):
    JOB_FILE = create_runner(script_path, 'Group', contrast_one)
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.dir = {'%s'}\n"%(output_path))
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.des.t1.scans = {\n")
    for con1 in flist: 
        JOB_FILE.write("                                                          '" + con1 + ",1'\n")
    JOB_FILE.write('                                                          };\n')

    JOB_FILE.write("""
matlabbatch{1}.spm.stats.factorial_design.cov = struct('c', {}, 'cname', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.multi_cov = struct('files', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.masking.tm.tm_none = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.im = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.em = {''};
matlabbatch{1}.spm.stats.factorial_design.globalc.g_omit = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.gmsca.gmsca_no = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.glonorm = 1;""")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.spmmat = {'%s'};\n"%(output_path+'/SPM.mat'))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.write_residuals = 0;\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.method.Classical = 1;\n")

    JOB_FILE.write("matlabbatch{3}.spm.stats.con.spmmat = {'%s'};\n"%(output_path+'/SPM.mat'))
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.name = '%s';\n"%( 1,contrast_one) )
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.weights = %s;\n"%(1,'[1]') )
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.sessrep = 'none';\n"%(1) )
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.delete = 1;\n")

    JOB_FILE.close()

In [32]:
parameters = []
for i,con in enumerate(Contrast):
    parameters.append( [con, Output_PATH + '/*/con_00%02d.nii'%(i+1) ]); mkdir(Second_level_output+'/'+con); mkdir(Script_PATH+'/1sample/'+con)
    print(parameters[-1])
    One_Sample_t_TEST(glob(parameters[-1][1]), Script_PATH+'/1sample', Second_level_output+'/'+con, parameters[-1][0])

['2bk_body-0bk_body', '/data/kyh/Workspace/WM0/Output/*/con_0001.nii']
['2bk_faces-0bk_faces', '/data/kyh/Workspace/WM0/Output/*/con_0002.nii']
['2bk_places-0bk_places', '/data/kyh/Workspace/WM0/Output/*/con_0003.nii']
['2bk_tools-0bk_tools', '/data/kyh/Workspace/WM0/Output/*/con_0004.nii']
['2bk_cor-0bk_cor', '/data/kyh/Workspace/WM0/Output/*/con_0005.nii']


In [105]:
contrast_file1 = sorted(glob(Output_PATH + '/*/con_0005.nii'))
contrast_file2 = sorted(glob(Output_PATH + '/*/con_0002.nii'))

Contrast_one = { #One sample only
    '2bk_cor-0bk_cor': '[ 1 ]'
}


Contrast_two = { # Two sample only
        'disc-detect' : '[1 -1]',
        'detect-disc' : '[-1 1]'
}

print('Contrast #1:', len(contrast_file1))
print('Contrast #2:', len(contrast_file2))

Contrast #1: 774
Contrast #2: 774


In [56]:
for parameter in parameters:
    print(parameter)

['lf-rf', '/data/kyh/Workspace/MOTOR0/Output/*/con_0001.nii']
['lh-rh', '/data/kyh/Workspace/MOTOR0/Output/*/con_0002.nii']
['lf-lh', '/data/kyh/Workspace/MOTOR0/Output/*/con_0003.nii']
['rf-rh', '/data/kyh/Workspace/MOTOR0/Output/*/con_0004.nii']
['t-lf', '/data/kyh/Workspace/MOTOR0/Output/*/con_0005.nii']
['t-lh', '/data/kyh/Workspace/MOTOR0/Output/*/con_0006.nii']
['t-rf', '/data/kyh/Workspace/MOTOR0/Output/*/con_0007.nii']
['t-rh', '/data/kyh/Workspace/MOTOR0/Output/*/con_0008.nii']


In [106]:
def One_Sample_t_TEST():
    global Script_PATH, Second_level_output, contrast_file1, Contrast_one
    JOB_FILE = create_runner(Script_PATH, 'Group', 'Group_analysis_1_Sample')
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.dir = {'%s'}\n"%(Second_level_output))
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.des.t1.scans = {\n")
    for con1 in contrast_file1: 
        JOB_FILE.write("                                                          '" + con1 + ",1'\n")
    JOB_FILE.write('                                                          };\n')

    JOB_FILE.write("""
matlabbatch{1}.spm.stats.factorial_design.cov = struct('c', {}, 'cname', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.multi_cov = struct('files', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.masking.tm.tm_none = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.im = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.em = {''};
matlabbatch{1}.spm.stats.factorial_design.globalc.g_omit = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.gmsca.gmsca_no = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.glonorm = 1;""")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.spmmat = {'%s'};\n"%(Second_level_output+'/SPM.mat'))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.write_residuals = 0;\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.method.Classical = 1;\n")

    JOB_FILE.write("matlabbatch{3}.spm.stats.con.spmmat = {'%s'};\n"%(Second_level_output+'/SPM.mat'))
    for Name, seq in zip(Contrast_one, range(1,len(Contrast_one)+1)):
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.name = '%s';\n"%(seq,Name) )
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.weights = %s;\n"%(seq,Contrast_one[Name]) )
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.sessrep = 'none';\n"%(seq) )
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.delete = 1;\n")

    JOB_FILE.close()
    
def Two_Sample_t_TEST():
    global Script_PATH, Second_level_output,contrast_file1,contrast_file2, Contrast_two
    JOB_FILE = create_runner(Script_PATH, 'Group', 'Group_analysis_2_Sample')
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.dir = {'%s'}\n"%(Second_level_output))

    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.des.t2.scans1 = {\n")
    for con1 in contrast_file1: 
        JOB_FILE.write("                                                          '" + con1 + ",1'\n")
    JOB_FILE.write('                                                          };\n')
    JOB_FILE.write("matlabbatch{1}.spm.stats.factorial_design.des.t2.scans2 = {\n")

    for con2 in contrast_file2: 
        JOB_FILE.write("                                                          '" + con2 + ",1'\n")
    JOB_FILE.write('                                                          };\n')
    JOB_FILE.write("""
matlabbatch{1}.spm.stats.factorial_design.des.t2.dept = 0;
matlabbatch{1}.spm.stats.factorial_design.des.t2.variance = 1;
matlabbatch{1}.spm.stats.factorial_design.des.t2.gmsca = 0;
matlabbatch{1}.spm.stats.factorial_design.des.t2.ancova = 0;
matlabbatch{1}.spm.stats.factorial_design.cov = struct('c', {}, 'cname', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.multi_cov = struct('files', {}, 'iCFI', {}, 'iCC', {});
matlabbatch{1}.spm.stats.factorial_design.masking.tm.tm_none = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.im = 1;
matlabbatch{1}.spm.stats.factorial_design.masking.em = {''};
matlabbatch{1}.spm.stats.factorial_design.globalc.g_omit = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.gmsca.gmsca_no = 1;
matlabbatch{1}.spm.stats.factorial_design.globalm.glonorm = 1;""")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.spmmat = {'%s'};\n"%(Second_level_output+'/SPM.mat'))
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.write_residuals = 0;\n")
    JOB_FILE.write("matlabbatch{2}.spm.stats.fmri_est.method.Classical = 1;\n")

    JOB_FILE.write("matlabbatch{3}.spm.stats.con.spmmat = {'%s'};\n"%(Second_level_output+'/SPM.mat'))
    for Name, seq in zip(Contrast_two, range(1,len(Contrast_two)+1)):
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.name = '%s';\n"%(seq,Name) )
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.weights = %s;\n"%(seq,Contrast_two[Name]) )
        JOB_FILE.write("matlabbatch{3}.spm.stats.con.consess{%d}.tcon.sessrep = 'none';\n"%(seq) )
    JOB_FILE.write("matlabbatch{3}.spm.stats.con.delete = 1;\n")

    JOB_FILE.close()

In [107]:
if   Mode=='one_sample': One_Sample_t_TEST()
elif Mode=='two_sample': Two_Sample_t_TEST()